Dataset

In [14]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from chess import pgn
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

In [15]:
def load_pgn(file_path):
    games = []
    with open(file_path) as file:
        while True:
            game = pgn.read_game(file)
            if game is None:
                break
            games.append(game)
    return games


def load_all_pgns(files):
    games = []
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(load_pgn, files), total=len(files)))
        for result in results:
            games.extend(result)
    return games

files = [file for file in os.listdir("data") if file.endswith(".pgn")]
LIMIT_OF_FILES = min(26, len(files))

games = load_all_pgns([f"data/{file}" for file in files[:LIMIT_OF_FILES]])
del files
print(f"Loaded {len(games)} games")

100%|██████████| 26/26 [02:33<00:00,  5.91s/it]

Loaded 19330 games


In [17]:
from dataset import ChessDataset
from model import ChessModel
from auxiliary_func import create_nn_input

_, _, move_to_int = create_nn_input(games)
num_classes = len(move_to_int)
print(f"Number of classes (unique moves): {num_classes}")

# Create Dataset and DataLoader
dataset = ChessDataset(games)
del games
dataloader = DataLoader(dataset, batch_size=64, shuffle=True) 

# Check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Model Initialization
num_classes = dataset.__getitem__
print(f"Number of classes: {num_classes}")
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

ValueError: not enough values to unpack (expected 3, got 2)

In [5]:
num_epochs = 100
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()

        outputs = model(inputs)  # Raw logits

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        running_loss += loss.item()
    end_time = time.time()
    epoch_time = end_time - start_time
    minutes: int = int(epoch_time // 60)
    seconds: int = int(epoch_time) - minutes * 60
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(dataloader):.4f}, Time: {minutes}m{seconds}s')

  0%|          | 0/24642 [00:00<?, ?it/s]

100%|██████████| 24642/24642 [07:18<00:00, 56.25it/s] 


Epoch 1/100, Loss: 3.9568, Time: 7m18s


100%|██████████| 24642/24642 [07:07<00:00, 57.71it/s] 


Epoch 2/100, Loss: 2.9318, Time: 7m7s


...

Epoch 98/100, Loss: 1.0573, Time: 2m19s


100%|██████████| 24642/24642 [02:17<00:00, 178.74it/s]


Epoch 99/100, Loss: 1.0540, Time: 2m17s


100%|██████████| 24642/24642 [02:16<00:00, 180.62it/s]

Epoch 100/100, Loss: 1.0507, Time: 2m16s


In [6]:
# Save the model
torch.save(model.state_dict(), "models/TORCH_100EPOCHS.pth")
import pickle

with open("models/100_move_to_int", "wb") as file:
    pickle.dump(move_to_int, file)